In [ ]:
import sys

parent_dir = 'Predict-Future-Sales'
p_sub = sys.path[0]

ride = ''
for path in p_sub.split('/'):
    if path != parent_dir:
        ride = ride + path + '/'
    else:
        ride = ride + path + '/'
        break
sys.path[0] = ride

import time
import pickle
import datetime

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from module.lino_module.preprocess import mode_of_freq, tde_dataset_wm
from module.lino_module.inference import RecurrentInference

from typing import Tuple
from torch import Tensor
from torch.utils.data import DataLoader

In [ ]:
from module.lino_module.model import TransformerModel

class WithAuxiliary(nn.Module):
    def __init__(self, d_model, head, device):
        super(WithAuxiliary, self).__init__()
        self.base = TransformerModel(d_model, head, device)
        self.auxiliary = TransformerModel(d_model, head, device)

    def forward(self, src, tgt, y=None):
        base_pred = self.base(src, tgt)
        auxiliary_pred = self.base(src, tgt)
        if self.training:
            auxiliary_label = y - base_pred.squeeze(2)
            return base_pred, auxiliary_pred, auxiliary_label
        else:
            return base_pred, auxiliary_pred

In [ ]:
class LossWithAuxiliary(nn.Module):
    def __init__(self, base_func, auxiliary_func):
        super(LossWithAuxiliary, self).__init__()
        self.base_func = base_func
        self.auxiliary_func = auxiliary_func

    def forward(self, base_pred, auxiliary_pred, y, auxiliary_label):
        base_loss = self.base_func(base_pred, y)
        auxiliary_loss = self.auxiliary_func(auxiliary_pred, auxiliary_label)
        loss = base_loss * 0.5 + auxiliary_loss * 0.5
        return loss, base_loss, auxiliary_loss

In [ ]:
def training_with_auxiliary(model: object,
                            train: DataLoader,
                            test: DataLoader,
                            device: torch.device,
                            criterion: object,
                            optimizer: object,
                            epochs: int,
                            verbose=10,
                            center=80) -> Tuple[object, Tensor, Tensor]:
    """訓練用関数"""
    loss_pack = {'train': {'loss': [], 'base': [], 'auxiliary': []},
              'eval': {'loss': [], 'base': [], 'auxiliary': []},
              'test': {'loss': [], 'base': [], 'auxiliary': []}}
    test_loss = []
    print(' start '.center(center, '-'))
    start_point = time.time()
    for epoch in range(epochs):
        epoch_loss = {'train': {'loss': [], 'base': [], 'auxiliary': []},
                      'eval': {'loss': [], 'base': [], 'auxiliary': []},
                      'test': {'loss': [], 'base': [], 'auxiliary': []}}

        cache = None
        for i, pack in enumerate(train):
            inputs = [content.to(device) for content in pack]
            # モデル訓練
            if i == 0:
                pass
            else:
                # キャッシュから１バッチ前のデータで訓練
                traing_eval(model, optimizer, criterion, cache, epoch_loss, 'train')
                # 勾配計算
                loss.backward()
                optimizer.step()

            # モデル評価
            traing_eval(model, optimizer, criterion, inputs, epoch_loss, 'eval')
            # データをキャッシュに保存して次回の訓練データにする
            cache = (src, tgt, y)

        # テストデータによる評価
        for pack in test:
            src, tgt, y = [content.to(device) for content in pack]
            traing_eval(model, optimizer, criterion, inputs, epoch_loss, 'test')

        # 損失データの登録
        appender(loss_pack, epoch_loss, 'train')
        appender(loss_pack, epoch_loss, 'eval')
        appender(loss_pack, epoch_loss, 'test')

        # lossのログを表示
        logger(verbose, epoch, center, epoch_loss)

    print(' complete!! '.center(center, '-'))
    print(f'Execution_time: {round(time.time() - start_point, 3)}')
    return model, loss_pack


In [1]:
def traing_eval(model, optimizer, criterion, inputs, epoch_loss, mode):
    src, tgt, y = inputs
    key = mode
    if mode=='test':
        mode = 'eval'
    getattr(model, mode)()
    model.train()
    optimizer.zero_grad()
    base, auxiliary, label = model(src, tgt, y)
    base, auxiliary = base.squeeze(2), auxiliary.squeeze(2)
    loss, base_loss, auxiliary_loss = criterion(base, auxiliary, y, label)
    output_pack = (base, auxiliary)

    epoch_loss[key]['loss'].append(loss.item())
    epoch_loss[key]['base'].append(base.item())
    epoch_loss[key]['auxiliary'].append(auxiliary.item())
    return output_pack

In [ ]:
def appender(dic1, dic2, mode):
    for key, value in dic2[mode].items():
        dic1[mode][key].append(value)
    return None

In [ ]:
def logger(verbose, epoch, center, epoch_loss):
    if verbose == 0:
        return None
    elif epoch % verbose == 0:
        print(f' epoch_{epoch} '.center(center))
        train_mean = torch.mean(torch.tensor(epoch_loss['train'])).item()
        valid_mean = torch.mean(torch.tensor(epoch_loss['eval'])).item()
        test_mean = torch.mean(torch.tensor(epoch_loss['test'])).item()
        print('train_loss: ', round(train_mean, 4),
                '| validation_loss: ', round(valid_mean, 4),
                '| test_loss: ', round(test_mean, 4))
        return None

In [ ]:
parent_path = '../../'
data = pd.read_csv(parent_path + '/data/sales_train.csv')
data = mode_of_freq(data).item_cnt_day
demo = np.arange(len(data))
ds = pd.Series(demo , index=data.index)

d_model = 4
dilation = 1

kwrgs ={'data': ds,
        'seq': 7,
        'd_model': d_model,
        'dilation': dilation,
        'src_tgt_seq': (6, 3),
        'step_num': 2,
        'batch_size': 64,
        'scaler': None,
        'daily': False,
        'weekly': False,
        'weekly_num': False,
        'monthly': False,
        'train_rate': 0.9
        }
train, test = tde_dataset_wm(**kwrgs)
src, tgt, y = next(iter(train))
src.shape, tgt.shape, y.shape

# 以下途中

In [ ]:
device = torch.device('cpu')
self = WithAuxiliary(d_model=4, head=2, device=device)
self.train()
base_pred, auxiliary_pred, label = self(src, tgt, y)
base_pred, auxiliary_pred = base_pred.squeeze(2), auxiliary_pred.squeeze(2)

loss_with_auxiliary = LossWithAuxiliary(nn.MSELoss(), nn.MSELoss())
loss, base_loss, auxiliary_loss = loss_with_auxiliary(base_pred, auxiliary_pred, y, label)

In [ ]:
x = torch.arange(24, dtype=torch.float).reshape(2, 3, 4)

size = (3, 4)
mask = torch.randn(size)

sigmoid = nn.Sigmoid()
norm = nn.LayerNorm(4)

normed = x@mask.T
atten_mask = sigmoid(normed)

atten_mask


class AttentionConv(nn.Module):
    def __init__(self, size, dim):
        super(AttentionConv, self).__init__()
        self.atten_weight = nn.rand(size)
        self.sigmoid = nn.Sigmoid()
        self.norm = nn.BatchNorm2d(dim)
    
    def forward(self, x):
        mask = 

import copy

class DifferPredictLayer(nn.Module):
    def __init__(self, module, n):
        super(DifferPredictLayer, self).__init__()
        # self.hidden = clones(module, n)
    
    def forward(self, data):
        return None
        
def clones(module, n): 
    return nn.Modulelist([copy.deepcopy(module) for _ in range(n)])